### Connect to GDrive

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Mar 13 08:03:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


### 1) Clone, configure & compile Darknet

In [4]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.16 MiB | 21.93 MiB/s, done.
Resolving deltas: 100% (10413/10413), done.


In [5]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile
!make

### Re-Train model to "person" class only

In [7]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
     
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [8]:
!echo -e 'person' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/CSE120Colab' > data/obj.data

In [9]:
# Download PASCAL VOC Dataset TEST AND TRAIN
!curl -O https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_06-Nov-2007.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  430M  100  430M    0     0  48.9M      0  0:00:08  0:00:08 --:--:-- 48.3M
--2023-03-13 08:05:49--  https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/octet-stream]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  26.5MB/s    in 9.8s    

2023-03-13 08:05:59 (44.8 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]



In [10]:
# Get weight file (can use same weights for ['person'] config)
!wget https://pjreddie.com/media/files/yolov3.weights
!mv yolov3.weights yolov3_training.weights

--2023-03-13 08:06:04--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  24.5MB/s    in 10s     

2023-03-13 08:06:15 (23.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# download scripts
%cd ..
!cp drive/MyDrive/CSE120Colab/voc.data darknet/cfg
!cp drive/MyDrive/CSE120Colab/voc_label.py darknet/scripts
!cp drive/MyDrive/CSE120Colab/trimPath.py darknet/scripts
!cp drive/MyDrive/CSE120Colab/voc.names darknet/data

# load pic
!cp drive/MyDrive/CSE120Colab/farmer.jpg darknet/data
%cd darknet/

/content
/content/darknet


In [36]:
!python3 scripts/voc_label.py

cat: 2007_train.txt: No such file or directory
cat: 2007_val.txt: No such file or directory
cat: 2007_train.txt: No such file or directory
cat: 2007_val.txt: No such file or directory
cat: 2007_test.txt: No such file or directory


In [37]:
!python3 scripts/trimPath.py 2007_person_train.txt
!python3 scripts/trimPath.py 2007_person_test.txt
!python3 scripts/trimPath.py 2007_person_val.txt

In [49]:
# "Download pretrained convolutional weights" https://pjreddie.com/darknet/yolo/
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-03-13 08:32:55--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  44.0MB/s    in 4.0s    

2023-03-13 08:32:59 (39.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!./darknet detector train data/voc.data cfg/yolov3-voc.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 86961, rewritten_bbox = 0.216189 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.727809), count: 9, class_loss = 0.502910, iou_loss = 0.323164, total_loss = 0.826075 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.545879), count: 4, class_loss = 0.495076, iou_loss = 0.465800, total_loss = 0.960875 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 86974, rewritten_bbox = 0.216157 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.616894), count: 4, class_loss = 0.299828, iou_loss = 0.207891, total_loss = 0.507719 
v3 (mse

In [51]:
!./darknet detector train data/voc.data cfg/yolov3-voc.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000018, iou_loss = 0.000000, total_loss = 0.000018 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000039, iou_loss = 0.000000, total_loss = 0.000039 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
 total_bbox = 103420, rewritten_bbox = 0.252369 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.0

In [ ]:
!./darknet detect cfg/yolov3.cfg backup/yolov3_training_final.weights data/farmer.jpg
!cp predictions.jpg /mydrive/CSE120Colab/newPredictions.jpg

In [ ]:
# copy images from GDrive -> Colab environment
!cp /mydrive/CSE120Colab/farmer.jpg data/

# For off the shelf yolo
!./darknet detect cfg/yolov3.cfg yolov3.weights data/farmer.jpg

In [ ]:
#Save results img
!cp predictions.jpg /mydrive/CSE120Colab/predictions.jpg

##### Original notebook from
https://github.com/emasterclassacademy

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

**6) Create train.txt file**

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show